In [1]:
import os
import sys
import json

sys.path.insert(0, '../../../pipelines/tasks/mlb/')

from common.helpers.parsers import parse_game

In [2]:
dump_file = False

def reduce_to_template(description: str, entities: dict):

    for key in entities:

        if key in ['runs', 'outs']:
            continue

        if not key in entities:
            description = description.replace(value, '-', 1)
        else:
            value = entities[key]
            if isinstance(value, str):
                description = description.replace(value, '-', 1)

            if isinstance(value, int):
                description = description.replace(str(value), '-', 1)

            if isinstance(value, list):
                for item in value:
                    if isinstance(item, dict):
                        description = reduce_to_template(description, item)
                    else:
                        description = description.replace(str(item), '-', 1)

    return description

In [3]:
templates = set([])

for file in os.listdir('../../../data/mlb/pbp/'):
    with open(f'../../../data/mlb/pbp/{file}', 'r', encoding='UTF8') as pbp_input:
        game = json.load(pbp_input)

    for period in game['periods']:
        for event in period['events']:

            if not 'entities' in event:
                ## print(file)
                print('entities')
                print(event)
                print()
            else:
                templates.add(
                    reduce_to_template(event['desc'], event['entities'])
                )

                if 'issues' in event['entities']:
                    ## print(file)
                    print('issues')
                    print(event)
                    print()


issues
{'isScoringPlay': False, 'pitches': [{'coords': {'x': 155, 'y': 150}, 'velocity': 95, 'type': 'four-seam fb', 'order': 1, 'result': {'type': 'strike', 'outcome': 'strike looking', 'bases': [1, 0, 0]}, 'count': {'balls': 0, 'strikes': 0}}, {'coords': {'x': 120, 'y': 163}, 'velocity': 88, 'type': 'slider', 'order': 2, 'result': {'type': 'strike', 'outcome': 'strike swinging', 'bases': [1, 0, 0]}, 'count': {'balls': 0, 'strikes': 1}}, {'coords': {'x': 109, 'y': 209}, 'velocity': 87, 'type': 'slider', 'order': 3, 'result': {'type': 'strike', 'outcome': 'strike swinging', 'bases': [1, 0, 0]}, 'count': {'balls': 0, 'strikes': 2}}], 'score': {'away': 0, 'home': 1}, 'desc': 'Kirilloff struck out swinging, catcher Tom Murphy to first baseman Ty France.', 'entities': {'player': 'Kirilloff', 'type': 'struck out', 'outs': 1, 'order': ['catcher Tom Murphy', 'first baseman Ty France'], 'effort': 'swinging', 'issues': ['order']}, 'id': 10}



In [4]:
print(len(templates))
templates

262


{'- - -, - caught stealing -, - to -.',
 '- - -, - safe at - on - by catcher Haase.',
 '- - -, - safe at - on - by catcher Jeffers.',
 '- - -, - safe at - on - by catcher Sánchez.',
 '- - -, - stole -.',
 '- - -, - to -.',
 '- - -.',
 '- - and caught stealing -.',
 '- - at - on -.',
 '- - on strikes.',
 '- - to - (- feet), - scored and - scored.',
 '- - to - (- feet), - scored, - scored and - scored.',
 '- - to - (- feet), - scored.',
 '- - to - (- feet).',
 '- - to -, - - at -, - safe at - on - by second baseman Lopez.',
 '- - to -, - - at -, - to -.',
 '- - to -, - - at -.',
 '- - to -, - - stretching at -, - to -.',
 '- - to -, - - stretching at -.',
 '- - to -, - safe at - on - by - baseman Polanco, - - at second.',
 '- - to -, - safe at - on - by - baseman Urshela, - to third.',
 '- - to -, - safe at - on - by center fielder Celestino.',
 '- - to -, - safe at - on - by center fielder Isbel, - safe at - on -.',
 '- - to -, - safe at - on - by left fielder Kwan.',
 '- - to -, - safe

In [5]:
if dump_file:

    keys = set()
    text_instances = []

    for file in os.listdir('../../../data/mlb/pbp/'):

        with open(f'../../../data/mlb/pbp/{file}', 'r', encoding='UTF8') as pbp_input:
            game = json.load(pbp_input)

        for period in game['periods']:
            for event in period['events']:

                description = event['desc']
                if not description in keys:
                    text_instances.append((description, event['entities']))
                    keys.add(description)

    with open('../../../pipelines/tasks/mlb/common/tests/docs/desc_to_entities.json', 'w') as test_instances:
        json.dump(text_instances, test_instances, indent=2)